In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("DSPy")
mlflow.autolog()

load_dotenv("../.env")
dspy.configure(
    lm=dspy.LM(
        "gemini/gemini-2.5-flash-lite", temperature=0.5, cache=True, rollout_id=1
    )
)

COMPOUND = "Luminespib"

2025/09/11 10:28:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/09/11 10:28:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/09/11 10:28:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
efficacy_agent = CFEfficacyAgent(max_iters=3)
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

Prediction(
    trajectory={'thought_0': "The user wants to estimate the efficacy of Luminespib for reversing cardiac fibrosis. I should start by searching for information about Luminespib's known activities and properties, especially in the context of cardiac fibrosis. I will use `search_chembl_id` to get the ChEMBL ID for Luminespib, which will be useful for subsequent searches.", 'tool_name_0': 'search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': "I have found two ChEMBL IDs for Luminespib. CHEMBL252164 is the primary ID. I should now use this ID to get more information about the compound's properties and bioactivities. I will start by getting the compound's properties using `get_compound_properties`.", 'tool_name_1': 'get_compound_properties', 'tool_args_1': {'chembl_id': 'CHEMBL252164'}, 'observation_1': "Properties of CHEMBL2521

[Trace(trace_id=941d39600e0240fe84a9993d7c3982e5), Trace(trace_id=18202932e9bc4e18aac02d53798cdc8d), Trace(trace_id=3f722276129b4b5d8c7ad26d808ccb74), Trace(trace_id=2eb06df48a694325967b29319fc55a38), Trace(trace_id=b4cf2ab80d074e3fb050f43c76f240a8)]

In [4]:
tox_agent = ToxicityScreeningAgent(max_iters=3)
tox_results = tox_agent(compound_name=COMPOUND)
print(tox_results)

Prediction(
    trajectory={'thought_0': "The user is asking to estimate the toxicity of the compound Luminespib. To do this, I need to find information about its safety and potential toxicity. I will start by searching for the compound's ChEMBL ID. If found, I will use it to retrieve safety information.", 'tool_name_0': 'search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': 'I have found two ChEMBL IDs for Luminespib. I should investigate both to ensure I have comprehensive information. I will start by getting the safety summary for the first ID, CHEMBL3137334.', 'tool_name_1': 'get_safety_summary', 'tool_args_1': {'cid': 'CHEMBL3137334'}, 'observation_1': 'No safety data available for CID CHEMBL3137334', 'thought_2': 'Since no safety data was found for CHEMBL3137334, I will now try to get the safety summary for the second ChEMBL ID, C

[Trace(trace_id=9a3380fbc9b9411c9b8beb627851c2ab), Trace(trace_id=fdbf28f6a19e41318892e71ac9479e58), Trace(trace_id=2df2754ced50466498126936863765a7), Trace(trace_id=d589ece22ea54d149f01c8ce2ac54f91)]

In [5]:
prioritization_agent = CompoundPrioritizationAgent()
prioritization_results = prioritization_agent(compound_name=COMPOUND)
print(prioritization_results)

Prediction(
    reasoning="The efficacy assessment indicates a lack of direct experimental evidence for Luminespib's antifibrotic effects. The toxicity assessment states that no safety data was found. Without any positive efficacy signals or safety information, Luminespib cannot be prioritized for cardiac fibrosis therapeutic development.",
    priority_score=0.0,
    confidence=0.0
)


[Trace(trace_id=51adc6056e4d4f528173efdfde82ab20), Trace(trace_id=34241c3d892d4751ad0e44e49d92f378), Trace(trace_id=33a8c7ce5a334738af09d3abc77b3c8c)]